In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
def drawing_bb(folder_path, video_name):

    out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v') ,5, (1024, 1024))
    allFiles = os.listdir(folder_path)
    files = []
    for f in allFiles:
        if ".jpg" in f:
            if "._" not in f:
                files.append(f)
#     print(files)
    for data_file in sorted(files):
        loc = folder_path+data_file
        print(loc)
        image = cv2.imread(loc)
        resized = cv2.resize(image, (1024,1024))

        out.write(resized)

        
    out.release()

In [3]:
folder_path = '/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/'
video_name = 'trial.mp4'
drawing_bb(folder_path, video_name)

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106840.5886662_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106846.1886785_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106851.7280421_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106857.2656734_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106862.6395154_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106868.078213_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106873.5155861_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106879.022853_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106884.5274942_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617106889.9653893_count:0.jpg
/home/team19/Desktop/Axis_DL/Det

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107330.435231_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107336.0048132_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107341.6129122_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107347.2183695_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107352.722399_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107358.2937963_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107363.835144_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107369.2074668_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107374.7791636_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107380.4830365_count:0.jpg
/home/team19/Desktop/Axis_DL/Dete

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107799.6989458_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107805.304318_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107810.9413128_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107816.5477061_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107822.0853837_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107827.5926616_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107833.1310585_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107838.7710233_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107844.276313_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617107849.848046_count:0.jpg
/home/team19/Desktop/Axis_DL/Dete

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108271.7667994_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108277.3048933_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108282.9085433_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108288.4152126_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108293.7554803_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108299.295197_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108304.8996565_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108310.4395492_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108315.9418836_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108321.4487789_count:0.jpg
/home/team19/Desktop/Axis_DL/De

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108740.462683_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108745.968435_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108751.5754268_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108757.0800412_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108762.4527276_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108767.958907_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108773.4636586_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108778.9700313_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108784.475178_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617108789.9803212_count:0.jpg
/home/team19/Desktop/Axis_DL/Detec

...

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114427.268854_count:3.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114432.9758487_count:3.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114438.680357_count:3.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114444.3212578_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114449.9570043_count:3.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114455.7319152_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114461.436255_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114467.0412376_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114472.6133165_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114478.1859283_count:2.jpg
/home/team19/Desktop/Axis_DL/Dete

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114903.2078102_count:1.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114908.747401_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114914.320242_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114919.891606_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114925.4643524_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114931.000843_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114936.5745726_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114942.1137545_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114947.6833415_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617114953.2582803_count:2.jpg
/home/team19/Desktop/Axis_DL/Detec

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115377.0436614_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115382.5483065_count:1.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115388.1222575_count:1.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115393.6624546_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115399.2010431_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115404.7400703_count:1.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115410.378324_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115415.9161918_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115421.4882064_count:2.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115426.993657_count:1.jpg
/home/team19/Desktop/Axis_DL/Det

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115850.5809357_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115856.084744_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115861.5926616_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115867.0969996_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115872.6034184_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115878.1415405_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115883.6486819_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115889.2210371_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115894.7928069_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617115900.4961545_count:0.jpg
/home/team19/Desktop/Axis_DL/De

/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116324.2525964_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116329.8250756_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116335.497038_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116341.067183_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116346.6055381_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116352.2774618_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116357.7511606_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116363.3230507_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116368.830593_count:0.jpg
/home/team19/Desktop/Axis_DL/Detection/YOLO/zongxin_test/Camera 1/1617116374.3331425_count:0.jpg
/home/team19/Desktop/Axis_DL/Dete